In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-11-02"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
14866,2024-11-02,Brasil Nbb,17:00,Paulistano,Botafogo,1.12,5.66,150.5,1.80,1.86,-11.5,1.93,1.77,dGNPZ9Ii,0.892857,0.176678,0.555556,0.537634,0.069536,0.8,0.4,NaN,NaN,124.262,36.953244,0.297382,193.148,71.879343,0.372146,137.896,248.762,111.39,284.16,0.0,0.0,0.0,0.0,0.946981,0.023184,0.061155,-3.82,-0.764,-0.157068,0.651264,0.4,-0.251264,6.71,1.342,3.472429,0.291784,0.4,0.108216
14867,2024-11-02,Brasil Nbb,18:00,Bauru,São José,1.42,2.56,153.5,1.88,1.79,-6.5,2.02,1.67,QeFtWRHG,0.704225,0.390625,0.531915,0.558659,0.094850,0.8,0.8,NaN,NaN,190.874,34.960560,0.183160,194.968,113.458331,0.581933,156.508,222.430,224.20,174.80,0.0,0.0,0.0,0.0,0.405076,0.034681,0.134139,0.72,0.144,2.916667,0.672852,0.8,0.127148,-2.69,-0.538,-2.899628,0.461986,0.3,-0.161986
14868,2024-11-02,Brasil Nbb,18:00,Corinthians Paulista,Brasília,1.29,3.15,159.5,1.85,1.81,-8.5,1.96,1.72,hrHYXmm4,0.775194,0.317460,0.540541,0.552486,0.092654,0.4,0.4,NaN,NaN,204.056,160.437731,0.786244,227.560,128.544749,0.564883,148.738,289.960,145.80,233.18,0.0,0.0,0.0,0.0,0.592441,0.015456,0.092231,-5.00,-1.000,-0.290000,0.571481,0.2,-0.371481,-2.38,-0.476,-4.516807,0.258042,0.2,-0.058042
14869,2024-11-02,Espanha Acb,14:00,Joventut,Basquet Girona,1.26,3.90,159.5,1.82,1.92,-10.5,2.03,1.66,Ymw8Iv4L,0.793651,0.256410,0.549451,0.520833,0.050061,0.8,0.4,NaN,NaN,194.132,83.193689,0.428542,282.556,219.973793,0.778514,146.150,446.698,91.00,515.90,0.0,1.0,0.0,0.0,0.723551,0.037813,0.141805,2.05,0.410,0.634146,0.569787,0.8,0.230213,1.10,0.220,13.181818,0.000000,0.0,0.000000
14870,2024-11-02,Espanha Acb,16:45,Granada,Bilbao,2.07,1.76,157.5,1.86,1.88,1.5,1.92,1.88,6eQf9zR7,0.483092,0.568182,0.537634,0.531915,0.051274,0.8,0.4,NaN,NaN,266.454,143.991601,0.540399,226.776,80.803793,0.356315,165.906,215.792,201.00,235.17,0.0,0.0,0.0,0.0,0.114466,0.007563,0.014886,0.63,0.126,8.492063,0.483936,0.6,0.116064,-2.67,-0.534,-1.423221,0.327852,0.2,-0.127852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15098,2024-11-02,Noruega Blno,13:00,Asker,Nidaros Jets,1.48,2.53,169.5,1.87,1.79,-5.5,2.01,1.72,IwShsRu9,0.675676,0.395257,0.534759,0.558659,0.070933,0.4,0.4,NaN,NaN,443.420,223.046938,0.503015,516.014,218.053148,0.422572,340.032,360.360,123.76,516.52,0.0,1.0,0.0,0.0,0.370305,0.030912,0.109952,-2.35,-0.470,-1.021277,0.490717,0.3,-0.190717,-5.00,-1.000,-1.530000,0.327574,0.1,-0.227574
15099,2024-11-02,Polônia 1. Liga,14:00,HydroTruck Radom,MKKS Koszalin,1.24,3.52,160.5,1.85,1.81,-9.5,1.94,1.73,v3w3i7vP,0.806452,0.284091,0.540541,0.552486,0.090543,0.4,0.6,NaN,NaN,168.326,37.561031,0.223145,259.524,149.536957,0.576197,151.094,260.184,118.44,518.24,0.0,1.0,0.0,0.0,0.677396,0.015456,0.080922,-0.03,-0.006,-40.000000,0.547905,0.6,0.052095,-3.28,-0.656,-3.841463,0.378361,0.3,-0.078361
15100,2024-11-02,Suécia Liga De Basquete,10:00,Uppsala F,Lulea F,10.40,38.00,144.5,1.85,1.85,42.5,1.95,1.72,U9DMyxz8,0.096154,0.026316,0.540541,0.540541,-0.877530,0.0,1.0,NaN,NaN,748.948,984.250008,1.314177,143.174,53.594886,0.374334,0.000,956.380,2349.00,113.49,1.0,0.0,0.0,0.0,0.806452,0.000000,0.088629,0.00,0.000,inf,0.000000,0.0,0.000000,42.08,8.416,4.396388,0.000000,0.0,0.000000
15101,2024-11-02,Suécia Liga De Basquete,12:00,AIK Basket F,Lund F,1.41,2.62,153.5,1.83,1.83,-6.5,1.95,1.72,0G66uKzq,0.

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,19:00,Eua Nba,Charlotte Hornets,Boston Celtics,230.5,1.85,0.9679,Over
1,20:30,Eua Nba,Philadelphia 76ers,Memphis Grizzlies,220.5,1.85,0.9704,Over
2,20:30,Eua Nba,Toronto Raptors,Sacramento Kings,231.5,1.81,0.9707,Over
3,23:00,Eua Nba,Denver Nuggets,Utah Jazz,217.5,1.81,0.9864,Over
4,23:00,Eua Nba,Phoenix Suns,Portland Trail Blazers,224.5,1.93,0.9810,Over
